#### 데이터 구조
- AI-Hub의 감정 분류를 위한 대화 음성 데이터셋
    - https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=263
- 4차년도, 5차년도, 5차년도 2차라는 세 개의 데이터로 구성되어 있음
- 각 년도별로 wav 파일들과 wav 파일에 대한 정보를 담은 csv 파일 하나로 구성되어 있음
- csv 파일에는 wav파일들의 이름을 wav_id라는 컬럼에 기재하고 그에 대한 정보를 기록함
    - 감정 정보의 경우 라벨링을 담당한 다섯 사람이 하나의 wav파일에 들어있는 음성을 듣고 각각 7가지 감정(기쁨, 슬픔, 분노, 공포, 혐오, 중립, 놀라움) 중 하나로 판단하고 그 다섯 사람의 판단을 전부 기재

In [2]:
import pandas as pd

In [ ]:
#경고 및 완료확인용 beep 함수
def beepError():
    fr = 1000    # range : 37 ~ 32767
    du = 2000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
def beepSuccess():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

#### 1. 데이터 전처리

In [5]:
print(len(csv_4th)+len(csv_5th)+len(csv_5th2))

43991


In [4]:
csv_4th = pd.read_csv('e:/Data2/4차년도.csv',encoding='CP949')
csv_5th = pd.read_csv('e:/Data2/5차년도.csv',encoding='CP949')
csv_5th2 = pd.read_csv('e:/Data2/5차년도_2차.csv',encoding='CP949')

In [2]:
def emo_labeling(data):
    data.rename(columns = {'4번감정세기':'4번 감정세기'},inplace=True) # 다른 컬럼과 띄어쓰기 통일

    #감정 레이블 만드는 방법
    #데이터 설명에 보면 5인이 각각 라벨링한 것으로 되어 있으므로 1번 감정~5번 감정의 다수결로 감정을 결정
    data['감정'] = ''
    for i in range(len(data)):
        templist = [data.iloc[i]['1번 감정'],data.iloc[i]['2번 감정'],data.iloc[i]['3번 감정'],data.iloc[i]['4번 감정'],data.iloc[i]['5번 감정']]
        data.iloc[i,-1] = max(templist,key= templist.count)
    print('분노',list(data['감정']).count('Angry'),end=' ')
    print('슬픔',list(data['감정']).count('Sadness'),end=' ')
    print('중립',list(data['감정']).count('Neutral'),end=' ')
    print('혐오',list(data['감정']).count('Disgust'),end=' ')
    print('놀람',list(data['감정']).count('Surprise'),end=' ')
    print('공포',list(data['감정']).count('Fear'),end=' ')
    print('행복',list(data['감정']).count('Happiness'),end=' ')

In [ ]:
emo_labeling(csv_4th)
emo_labeling(csv_5th)
emo_labeling(csv_5th2)

In [ ]:
#다른 csv파일과 감정명 통일(두문자를 대문자로)

csv_5th2=csv_5th2.replace({'감정':'angry'}, 'Angry')
csv_5th2=csv_5th2.replace({'감정':'sadness'}, 'Sadness')
csv_5th2=csv_5th2.replace({'감정':'neutral'}, 'Neutral')
csv_5th2=csv_5th2.replace({'감정':'disgust'}, 'Disgust')
csv_5th2=csv_5th2.replace({'감정':'surprise'}, 'Surprise')
csv_5th2=csv_5th2.replace({'감정':'fear'}, 'Fear')
csv_5th2=csv_5th2.replace({'감정':'happiness'}, 'Happiness')

csv_5th2

In [ ]:
#필요한 정보만 담은 새로운 DF 만들기
def csv_df_ax1(data):
    result = pd.concat((data['wav_id'],data['발화문'],data['감정']),axis=1)
    return result

In [ ]:
csv_4th_get =csv_df_ax1(csv_4th)
csv_5th_get = csv_df_ax1(csv_5th)
csv_5th2_get = csv_df_ax1(csv_5th2)

csv_fin = pd.concat((csv_4th_get,csv_5th_get,csv_5th2_get))
csv_fin.to_csv('e:/Data2/fin_csv.csv',index=False) 

In [ ]:
#감정에 따른 DF로 나누기

emotion_list = ['Angry','Sadness','Neutral','Disgust','Surprise','Fear','Happiness']


for i in emotion_list:
    locals()[f'csv_{i}'] =csv_forwork[csv_forwork['감정'] == i]
    locals()[f'csv_{i}'].reset_index(drop=True,inplace=True)
print(len(csv_Neutral))
print(len(csv_Angry))
print(len(csv_Disgust)) 

In [ ]:
#감정 DF 다시 train,valid set으로 분리 후 합치기

# emotion_list = ['Angry','Sadness','Neutral','Disgust','Surprise','Fear','Happiness']


for i in emotion_list:
    locals()[f'csv_{i}_train'] =locals()[f'csv_{i}'].iloc[:len(locals()[f'csv_{i}'])//3*2,:]
    locals()[f'csv_{i}_train'].reset_index(drop=True,inplace=True)
    locals()[f'csv_{i}_valid'] =locals()[f'csv_{i}'].iloc[len(locals()[f'csv_{i}_train']):,:]
    locals()[f'csv_{i}_valid'].reset_index(drop=True,inplace=True)

train_csv = pd.concat((csv_Angry_train,csv_Sadness_train,csv_Neutral_train,csv_Disgust_train,csv_Surprise_train,
                       csv_Fear_train,csv_Happiness_train))
train_csv.reset_index(drop=True,inplace=True)

valid_csv = pd.concat((csv_Angry_valid,csv_Sadness_valid,csv_Neutral_valid,csv_Disgust_valid,csv_Surprise_valid,
                       csv_Fear_valid,csv_Happiness_valid))
valid_csv.reset_index(drop=True,inplace=True)

train_csv.to_csv('e:/Data2/train_csv.csv',index = False)
valid_csv.to_csv('e:/Data2/valid_csv.csv',index = False)

### 트러블 발생 (2)
- 파일 개수와 csv 정보의 수는 일치하는데 csv에는 기재되어 있지만 실존하지 않는 파일 발견
- 확인 결과 csv상의 이름과 실제 파일명이 다른 16개의 파일 발견
- csv 기록에 없는 파일을 직접 재생하여 듣고 그 내용이 csv 기록의 '발화문'과 일치하는 row의 'wav_id' 항목으로 파일명 변경

In [3]:
a = list(valid_csv['wav_id']) #valid df에 있는 파일명(.wav 제외)
for i in range(len(glob.glob(audio_path+'*.wav'))): #전체 파일의 개수만큼의 회수
    try:
        if os.listdir('e:/Data2/audio/')[0][:-4] in a: # a 리스트에 있는 것과 일치하는 파일명이 나온다면
            #파일이 이동하면서 사전 설정된 반복회수에 비해 파일 수가 줄어드므로 그냥 0번에서 다 분류해버리게 설정
            shutil.move(f"e:/Data2/audio/{os.listdir('e:/Data2/audio/')[0]}", f"e:/Data2/valid/{os.listdir('e:/Data2/audio/')[0]}")
            print('valid성공',end=' ')
        else:
            shutil.move(f"e:/Data2/audio/{os.listdir('e:/Data2/audio/')[0]}", f"e:/Data2/train/{os.listdir('e:/Data2/audio/')[0]}")
            print('train성공',end=' ') #일치하지 않는 파일은 train 폴더에
    except:
        beepError()

NameError: name 'features' is not defined

In [ ]:
train_csv = pd.read_csv('e:/Data2/csv/train.csv', encoding= 'utf-8')
valid_csv = pd.read_csv('e:/Data2/csv/valid.csv',encoding='utf-8')
csv_path = 'E:/Data2/csv/'
# vector_path = 'E:/Data2/vector/' #당장 안 쓰는 폴더
train_audio_path = 'E:/Data2/train/'
valid_audio_path = 'E:/Data2/valid/'
test_path = 'E:/Data2/test'

### 음성 특징 추출

In [ ]:
def extract_features(data):
    
    
    name_list = data['wav_id'] 
    name_list = ['5fbe364744697678c497c07f','5fbe361844697678c497c07e'] #테스트용
    all_features=[]
    timestep = 40

    try:
        for i in range(len(name_list)):
            y, sr = librosa.load(train_audio_path+name_list[i]+'.wav',sr=16000,res_type='kaiser_fast')
        #     y, sr = librosa.load('e:/Data2/test/'+name_list[i]+'.wav',sr=16000,res_type='kaiser_fast') #테스트용

            # 특성들 추출
            #(특성, 시간) 형태
            #시간은 같을 테니까 뒤집어 주고 패딩하기
            mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13)
            spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            spec_cont = librosa.feature.spectral_contrast(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            spec_flat = librosa.feature.spectral_flatness(y=y)
            inharmonicity = librosa.feature.spectral_centroid(y=y, sr=sr)
            harmonic, percussive = librosa.effects.hpss(y)
            harmonic_energy = librosa.feature.rms(y=harmonic)
            noise_energy = librosa.feature.rms(y=percussive)
            # chroma 차원 확장하여 concat 가능한 형태로 변환
            C = np.abs(librosa.cqt(y, sr=sr, bins_per_octave=12, n_bins=7*12))
            # 크로마 CQT 스펙트럼을 크로마 그램으로 변환
            chroma = librosa.feature.chroma_cqt( C=C,n_chroma=12)

            mfcc = np.vstack((mfcc,spec_rolloff))
            mfcc = np.vstack((mfcc,spec_cont))
            mfcc = np.vstack((mfcc,spec_cent))
            mfcc = np.vstack((mfcc,spec_bw))
            mfcc = np.vstack((mfcc,spec_flat))
            mfcc = np.vstack((mfcc,inharmonicity))
            mfcc = np.vstack((mfcc,harmonic_energy))
            mfcc = np.vstack((mfcc,noise_energy))
            mfcc = np.vstack((mfcc,chroma))
            mfcc = pad_sequences(mfcc, maxlen=timestep, padding='post',dtype='float32')

            mfcc = mfcc.T


            mfcc = pad_sequences(mfcc, maxlen=13, padding='post',dtype='float32')

            mfcc = np.pad(mfcc, ((0,0),(2,2)), 'constant', constant_values=0)
            all_features.append(mfcc)
            if i>0 and i%100==0:
                print(i,'번째 파일 완료')

        all_features = np.array(all_features).reshape(-1,40,17) #테스트용
        beepSuccess()
    except:
        beepError()

    return all_features


In [ ]:
features = extract_features(train_csv)
np.save('./features',features)

In [ ]:
y= pd.get_dummies(valid_csv['감정'],dtype=int)
valid_csv = pd.concat((valid_csv, y),axis =1)
y= pd.get_dummies(train_csv['감정'],dtype=int)
train_csv = pd.concat((train_csv, y),axis =1)
valid_csv.reset_index(drop=True,inplace=True)
train_csv.reset_index(drop=True,inplace=True)
train_csv.to_csv('e:/Data2/train_csv.csv',index = False)
valid_csv.to_csv('e:/Data2/valid_csv.csv',index = False)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten

def first_model(x_train):
    model=Sequential()
    model.add(InputLayer(shape=(58,1)))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=7, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #model.summary()
    return model

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#스케일링 - 데이터를 평균 0, 표준편차 1로 스케일을 축소
scaler = StandardScaler()

scaler.fit(train_data)
   
X_train, X_test, y_train, y_test = train_test_split(train_data,y, random_state=0,  test_size = 0.3)



X_train.shape, y_train.shape, X_test.shape, y_test.shape

model = first_model(X_train)
history=model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test), callbacks=[rlrp,mcp,es])

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Pre-trained Model: ", i)
print("Test Accuracy: ",test_acc)

#결과 accuracy도 test accuracy도 약 45% 전후로 나옴